### Recommendation System via a Gaussian Mixture Model for Matrix Completion

We build the entire model using only Numpy.

In [1]:
import numpy as np

First we import our partially observed data matrix X along with the complete matrix which we will use as the ground truth to compare to later.

In [2]:
X = np.loadtxt('netflix_incomplete.txt')
X_complete = np.loadtxt('netflix_complete.txt')

Next we extract parameters from our matrix and initialize the mixture which is defined by mu (mean), p (weight), and var(variance). 

In [3]:
#parameters
n, d = X.shape
K = 12
delta = np.where(X,0,1)

#mixture
np.random.seed(1)
mu = X[np.random.choice(n,K, replace = False)]
p = np.ones(K)/K
var = np.sum((mu*np.ones([n,K,d]) - X.reshape([n,1,d]))**2, axis=(0,2))/(n*d)

Next we define a function for computing the squared norm which depends only on the current mu.

In [4]:
def compute_norm(mu):
    U = (mu*np.ones([n,K,d]))*delta.reshape([n,1,d])
    sub_stack = U - X.reshape([n,1,d])
    return np.sum(sub_stack**2, axis = 2)

Next, we define a function which runs the E-Step of the EM algorithm returning the soft counts (posterior) and the log-likelihood of the assignment.

In [5]:
def estep(X,mu,p,var):
    
    norm = compute_norm(mu)
    
    C_u = np.sum(delta,axis=1,keepdims=True)
    logged_gauss = np.log(p) - C_u/2*np.log(2*np.pi*var*np.ones([n,K])) - norm/(2*var)
    max_vector = np.amax(logged_gauss, axis=1, keepdims=True)
    scaled_gauss = np.exp(logged_gauss - max_vector)
    denom = max_vector + np.log(np.sum(scaled_gauss, axis=1, keepdims=True))
    
    log_post = logged_gauss - denom
    log_likelihood = np.sum(denom)
    
    return np.exp(log_post), log_likelihood

Next we define a function which runs the M-Step of the EM algorithm returning the updated mixture.

In [6]:
def mstep(X, post, min_var, mu, p, var):
    
    norm = compute_norm(mu)
    
    #update mu
    mu_numer = np.dot(X.T, post).T
    mu_denom = np.dot(delta.T, post).T
    mu = np.where(mu_denom >= 1, mu_numer/(mu_denom +1e-10), mu)
    
    #update var
    C_u = np.sum(delta, axis=1, keepdims=True)
    sum_factor = np.sum(post*norm, axis = 0)
    first_factor = 1/np.sum(C_u*post, axis = 0)
    var_bad = first_factor*sum_factor
    var = np.where(var_bad < min_var, min_var, var_bad)
    
    #update p
    p = np.sum(post, axis = 0)/n
    
    #update norm
    norm = compute_norm(mu)
    
    return mu,p,var

Next, we define a function which runs the entire EM algorithm using our E-Step and M-Step functions.

In [7]:
history = []

def run(X, mu, p ,var, min_var=.1):
    
    old_log = None
    new_log = None
    while old_log is None or (new_log - old_log > 1e-6*np.abs(new_log)):
        old_log=new_log
        
        #E-step
        post, new_log = estep(X, mu, p, var)
        
        #M-step
        mu, p, var = mstep(X, post, min_var, mu, p, var)
    
    return mu,p,var,post,new_log
        
mu,p,var,post,old_log = run(X, mu, p, var, 0.1)

Next, we use the mixture to "fill" our incomplete matrix.

In [8]:
def fill(X,mu,p,var):
    
    post,_ = estep(X,mu,p,var)
    X_pred = X.copy()
    miss_indices = np.where(X == 0)
    X_pred[miss_indices] = (post@mu)[miss_indices]
    
    return X_pred

X_filled = fill(X,mu,p,var)

Finally we calculate the rmse between the complete and incomplete matrix. (It is important to know that the completed matrix was not used at all up to this point as in real world applications, we would not have access to that information.)

In [9]:
def rmse(X, Y):
    return np.sqrt(np.mean((X - Y)**2))

rmse(X_filled, X_complete)

1.051719790109855

In [10]:
rmse(X, X_complete)

1.6787480867863673

The rmse score is much lower now as we had hoped.